In [ ]:
#default_exp data.datasets.tourism

# Tourism dataset

> API details.

In [ ]:
#export
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Optional, Tuple, Union

import numpy as np
import pandas as pd
from pandas.tseries.frequencies import to_offset

from nixtla.data.datasets.utils import download_file, Info, TimeSeriesDataclass
from nixtla.data.ts_dataset import TimeSeriesDataset

In [ ]:
#export
SOURCE_URL = 'https://robjhyndman.com/data/27-3-Athanasopoulos1.zip'

Tourism meta information

In [ ]:
#export
@dataclass
class Yearly:
    seasonality: int = 1
    horizon: int = 4
    freq: str = 'Y'
    rows: int = 2
    name: str = 'Yearly'
    n_ts: int = 518

@dataclass
class Quarterly:
    seasonality: int = 4
    horizon: int = 8
    freq: str = 'Q'
    rows: int = 3
    name: str = 'Quarterly'
    n_ts: int = 427

@dataclass
class Monthly:
    seasonality: int = 12
    horizon: int = 24
    freq: str = 'M'
    rows: int = 3
    name: str = 'Monthly'
    n_ts: int = 366

In [ ]:
#export
TourismInfo = Info(groups=('Yearly', 'Quarterly', 'Monthly'),
                   class_groups=(Yearly, Quarterly, Monthly))

In [ ]:
#export
class Tourism(TimeSeriesDataclass):

    @staticmethod
    def load(directory: str,
             group: str,
             return_tensor: bool = True): #-> Union[TimeSeriesDataset, TimeSeriesDataclass]:
        """
        Downloads and loads Tourism data.

        Parameters
        ----------
        directory: str
            Directory where data will be downloaded.
        group: str
            Group name.
            Allowed groups: 'Yearly', 'Quarterly', 'Monthly'.
        return_tensor: bool
            Wheter return TimeSeriesDataset (tensors, True) or
            TimeSeriesDataclass (dataframes)
            
        Notes
        -----
        [1] Returns train+test sets.
        """
        path = Path(directory) / 'tourism' / 'datasets'

        Tourism.download(directory)

        class_group = TourismInfo.get_group(group)

        train_file = path / f'{class_group.name.lower()}_in.csv'
        test_file = path / f'{class_group.name.lower()}_oos.csv'
        
        train, test = pd.read_csv(train_file), pd.read_csv(test_file)

        dfs = []
        freq = to_offset(class_group.freq)
        for col in train.columns:
            df_appended = []
            for df, training in zip([train, test], [True, False]):
                df_col = df[col]
                length, year = df_col[:2].astype(int)
                skip_rows = class_group.rows
                start_date = pd.to_datetime(f'{year}-01-01')
                if group != 'Yearly':
                    n_offsets = df_col[2].astype(int)
                    start_date += n_offsets * freq
                elif col == 'Y18' and not training: # viene mal en el archivo esta serie
                    start_date += 2 * freq
                df_col = df_col[skip_rows:length + skip_rows]
                df_col = df_col.rename('y').to_frame()
                df_col['unique_id'] = col
                df_col['ds'] = pd.date_range(start_date, periods=length, freq=freq)
                df_appended.append(df_col)
            df_appended = pd.concat(df_appended)
            dfs.append(df_appended)

        df = pd.concat(dfs)

        df = df.reset_index().filter(items=['unique_id', 'ds', 'y'])
        df = df.sort_values(['unique_id', 'ds'])
        
        #if return_tensor:
        #    return TimeSeriesDataset(y_df=df, X_s_df=None, X_t_df=None, output_size=class_group.horizon)
        #else:
        #    return TimeSeriesDataclass(Y=df, S=None, X=None, group=group)
        return df, None

    @staticmethod
    def download(directory: str) -> None:
        """Downloads Tourism Dataset."""
        path = Path(directory) / 'tourism' / 'datasets'
        if not path.exists():
            download_file(path, SOURCE_URL, decompress=True)

In [ ]:
for group in TourismInfo.groups:
    print(group)
    tourism_dataset = Tourism.load(directory='data', group=group)
    print(tourism_dataset.ts_tensor)

Yearly
Processing dataframes ...
Creating ts tensor ...
tensor([[[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 3.7386e+04,
          3.8432e+04, 4.0345e+04],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 1.0000e+00,
          1.0000e+00, 1.0000e+00]],

        [[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 3.9105e+05,
          4.5404e+05, 5.5294e+05],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 1.0000e+00,
          1.0000e+00, 1.0000e+00]],

        [[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 1.8051e+03,
          2.1386e+03, 2.6821e+03],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 1.0000e+00,
          1.0000e+00, 1.0000e+00]],

        ...,

        [[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 4.8804e+00,
          5.3868e+00, 5.7406e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 1.0000e+00,
          1.0000e+00, 1.0000e+00]],

        [[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 2.3240e+01,
          2.5290e+01, 2.7336e+01],
         [0.0000e+00, 0.0000e+0

In [ ]:
for group in TourismInfo.class_groups:
    data = Tourism.load(directory='data', group=group.name, return_tensor=False).Y
    unique_elements = data.groupby(['unique_id', 'ds']).size()
    unique_ts = data.groupby('unique_id').size()

    assert (unique_elements != 1).sum() == 0, f'Duplicated records found: {group.name}'
    assert unique_ts.shape[0] == group.n_ts, f'Number of time series not match: {group.name}'